Detección de caras con webcam

In [1]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [3]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(1)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 0
imodoE = 0

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    # Face detector change
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    #Eye detector change
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
1/1 [==============================] - 0s 20ms/step


In [5]:
import cv2
import numpy as np
import time
import random

normalizatorHS = faceutils.Normalization()
FDet = FaceDetectors.FaceDetector()

font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap is not None:
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

cap.set(3, 640)
cap.set(4, 480)

imodoF = 0
imodoE = 0
score = 0

bird_y = 240
bird_velocity = 0
gravity = 1
game_over = False
game_started = False
space_pressed = False

obstacle_x = 640
obstacle_speed = 5
obstacle_width = 50
obstacle_height_upper = 100  # Altura del obstáculo superior
obstacle_height_lower = 200  # Altura del obstáculo inferior

def reset_game():
    global bird_y, bird_velocity, score, game_over, game_started, obstacle_x
    bird_y = 240
    bird_velocity = 0
    score = 0
    game_over = False
    game_started = False
    obstacle_x = 640

reset_game()

while True:
    t = time.time()
    ret, frame = cap.read()
    B, G, R = cv2.split(frame)

    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values
        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

                if not game_over:
                    if game_started:
                        bird_y = ley

                        obstacle_x -= obstacle_speed
                        if obstacle_x < -obstacle_width:
                            obstacle_x = 640
                            # Altura aleatoria de los obstáculos
                            obstacle_height_upper = random.randint(50, 250)
                            obstacle_height_lower = 350 - obstacle_height_upper
                            score += 1

                        cv2.circle(frame, (100, bird_y), 20, (0, 255, 0), -1)
                        # Obstáculo superior
                        cv2.rectangle(frame, (obstacle_x, 0), (obstacle_x + obstacle_width, obstacle_height_upper), (0, 0, 255), -1)
                        # Obstáculo inferior
                        cv2.rectangle(frame, (obstacle_x, 480 - obstacle_height_lower), (obstacle_x + obstacle_width, 480), (0, 0, 255), -1)

                        # Detección de colisión con los obstáculos
                        bird_x = 100
                        bird_radius = 20
                        if (
                            bird_x + bird_radius > obstacle_x
                            and bird_x - bird_radius < obstacle_x + obstacle_width
                            and (bird_y - bird_radius < obstacle_height_upper or bird_y + bird_radius > 480 - obstacle_height_lower)
                        ):
                            game_over = True

                    if not game_started:
                        cv2.putText(frame, "Press 'Space' to Start", (200, 200), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
                else:
                    cv2.putText(frame, "Game Over", (280, 200), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

                cv2.putText(frame, "Score: " + str(score), (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Cam', frame)

    key = cv2.waitKey(40)
    if key == 27:  # Tecla Esc para salir
        break
    elif key == 32:  # Tecla Espacio
        if not game_started:
            if not space_pressed:
                reset_game()
                game_started = True
                space_pressed = True
        else:
            space_pressed = False

cap.release()
cv2.destroyAllWindows()


Camera 0


In [10]:
import cv2
import numpy as np
import time
import random

normalizatorHS = faceutils.Normalization()
FDet = FaceDetectors.FaceDetector()

font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap is not None:
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

cap.set(3, 640)
cap.set(4, 480)

imodoF = 0
imodoE = 0
score = 0
speed_increment_counter = 0
obstacle_speed = 5
obstacle_speed_increment = 2  # Aumento de velocidad cada 2 puntos
max_speed = 20  # Velocidad máxima de los obstáculos

bird_y = 240
bird_velocity = 0
gravity = 1
game_over = False
game_started = False
space_pressed = False

obstacles = []

obstacle_width = 50  # Ancho de los obstáculos

def reset_game():
    global bird_y, bird_velocity, score, game_over, game_started, obstacles, speed_increment_counter, obstacle_speed
    bird_y = 240
    bird_velocity = 0
    score = 0
    game_over = False
    game_started = False
    obstacles = []
    speed_increment_counter = 0
    obstacle_speed = 5

def generate_obstacle():
    x = 640
    upper_height = random.randint(50, 250)
    lower_height = 350 - upper_height
    return (x, upper_height, lower_height)

reset_game()

while True:
    t = time.time()
    ret, frame = cap.read()
    B, G, R = cv2.split(frame)

    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values
        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

                if not game_over:
                    if game_started:
                        bird_y = ley

                        # Mover y actualizar los obstáculos existentes
                        new_obstacles = []
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            x -= obstacle_speed
                            if x >= -obstacle_width:
                                new_obstacles.append((x, upper_height, lower_height))
                        obstacles = new_obstacles

                        # Generar nuevos obstáculos
                        if len(obstacles) == 0 or obstacles[-1][0] < 400:  # Generar un nuevo obstáculo cuando el último esté cerca del centro
                            obstacles.append(generate_obstacle())

                        # Detección de colisión con los obstáculos
                        bird_x = 100
                        bird_radius = 20
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            if (
                                bird_x + bird_radius > x
                                and bird_x - bird_radius < x + obstacle_width
                                and (bird_y - bird_radius < upper_height or bird_y + bird_radius > 480 - lower_height)
                            ):
                                game_over = True

                        cv2.circle(frame, (100, bird_y), 20, (0, 255, 0), -1)
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            cv2.rectangle(frame, (x, 0), (x + obstacle_width, upper_height), (0, 0, 255), -1)
                            cv2.rectangle(frame, (x, 480 - lower_height), (x + obstacle_width, 480), (0, 0, 255), -1)

                    if not game_started:
                        cv2.putText(frame, "Press 'Space' to Start", (200, 200), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
                else:
                    cv2.putText(frame, "Game Over", (280, 200), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

                cv2.putText(frame, "Score: " + str(score), (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Cam', frame)

    key = cv2.waitKey(40)
    if key == 27:  # Tecla Esc para salir
        break
    elif key == 32:  # Tecla Espacio
        if not game_started:
            if not space_pressed:
                reset_game()
                game_started = True
                space_pressed = True
        else:
            space_pressed = False

cap.release()
cv2.destroyAllWindows()


Camera 0


In [2]:
import cv2
import numpy as np
import time
import random

normalizatorHS = faceutils.Normalization()
FDet = FaceDetectors.FaceDetector()

font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap is not None:
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

cap.set(3, 640)
cap.set(4, 480)

imodoF = 0
imodoE = 0
score = 0
speed_increment_counter = 0
obstacle_speed = 5
obstacle_speed_increment = 2  # Aumento de velocidad cada 2 puntos
max_speed = 20  # Velocidad máxima de los obstáculos

bird_y = 240
bird_velocity = 0
gravity = 1
game_over = False
game_started = False
space_pressed = False
bird_passed_obstacle = False  # Variable para controlar el puntaje

obstacles = []

obstacle_width = 50  # Ancho de los obstáculos

def reset_game():
    global bird_y, bird_velocity, score, game_over, game_started, obstacles, speed_increment_counter, obstacle_speed
    bird_y = 240
    bird_velocity = 0
    score = 0
    game_over = False
    game_started = False
    obstacles = []
    speed_increment_counter = 0
    obstacle_speed = 5

def generate_obstacle():
    x = 640
    upper_height = random.randint(50, 250)
    lower_height = 350 - upper_height
    return (x, upper_height, lower_height)

reset_game()

while True:
    t = time.time()
    ret, frame = cap.read()
    B, G, R = cv2.split(frame)

    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values
        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

                if not game_over:
                    if game_started:
                        bird_y = ley

                        # Mover y actualizar los obstáculos existentes
                        new_obstacles = []
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            x -= obstacle_speed
                            if x >= -obstacle_width:
                                new_obstacles.append((x, upper_height, lower_height))
                        obstacles = new_obstacles

                        # Generar nuevos obstáculos
                        if len(obstacles) == 0 or obstacles[-1][0] < 400:  # Generar un nuevo obstáculo cuando el último esté cerca del centro
                            obstacles.append(generate_obstacle())

                        # Detección de colisión con los obstáculos
                        bird_x = 100
                        bird_radius = 20
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            if (
                                bird_x + bird_radius > x
                                and bird_x - bird_radius < x + obstacle_width
                                and (bird_y - bird_radius < upper_height or bird_y + bird_radius > 480 - lower_height)
                            ):
                                game_over = True

                        # Incrementar puntos cuando el obstáculo haya pasado completamente
                        if obstacles and obstacles[0][0] + obstacle_width < bird_x - bird_radius:
                            if not bird_passed_obstacle:
                                score += 1
                                bird_passed_obstacle = True
                        else:
                            bird_passed_obstacle = False

                        cv2.circle(frame, (100, bird_y), 20, (0, 255, 0), -1)
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            cv2.rectangle(frame, (x, 0), (x + obstacle_width, upper_height), (0, 0, 255), -1)
                            cv2.rectangle(frame, (x, 480 - lower_height), (x + obstacle_width, 480), (0, 0, 255), -1)

                    if not game_started:
                        cv2.putText(frame, "Press 'Space' to Start", (200, 200), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
                else:
                    cv2.putText(frame, "Game Over", (280, 200), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

                cv2.putText(frame, "Score: " + str(score), (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Cam', frame)

    key = cv2.waitKey(40)
    if key == 27:  # Tecla Esc para salir
        break
    elif key == 32:  # Tecla Espacio
        if not game_started:
            if not space_pressed:
                reset_game()
                game_started = True
                space_pressed = True
        else:
            space_pressed = False

cap.release()
cv2.destroyAllWindows()




Camera 0


In [3]:
import cv2
import numpy as np
import time
import random

normalizatorHS = faceutils.Normalization()
FDet = FaceDetectors.FaceDetector()

font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap is not None:
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

cap.set(3, 640)
cap.set(4, 480)

imodoF = 0
imodoE = 0
score = 0
speed_increment_counter = 0
obstacle_speed = 5
obstacle_speed_increment = 1 # Aumento de velocidad cada 2 puntos
max_speed = 20  # Velocidad máxima de los obstáculos

bird_y = 240
bird_velocity = 0
gravity = 1
game_over = False
game_started = False
space_pressed = False
bird_passed_obstacle = False  # Variable para controlar el puntaje

obstacles = []

obstacle_width = 50  # Ancho de los obstáculos

def reset_game():
    global bird_y, bird_velocity, score, game_over, game_started, obstacles, speed_increment_counter, obstacle_speed
    bird_y = 240
    bird_velocity = 0
    score = 0
    game_over = False
    game_started = False
    obstacles = []
    speed_increment_counter = 0
    obstacle_speed = 5

def generate_obstacle():
    x = 640
    upper_height = random.randint(50, 250)
    lower_height = 350 - upper_height
    return (x, upper_height, lower_height)

reset_game()

while True:
    t = time.time()
    ret, frame = cap.read()
    B, G, R = cv2.split(frame)

    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values
        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

                if not game_over:
                    if game_started:
                        bird_y = ley

                        # Mover y actualizar los obstáculos existentes
                        new_obstacles = []
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            x -= obstacle_speed
                            if x >= -obstacle_width:
                                new_obstacles.append((x, upper_height, lower_height))
                        obstacles = new_obstacles

                        # Generar nuevos obstáculos
                        if len(obstacles) == 0 or obstacles[-1][0] < 400:  # Generar un nuevo obstáculo cuando el último esté cerca del centro
                            obstacles.append(generate_obstacle())

                        # Detección de colisión con los obstáculos
                        bird_x = 100
                        bird_radius = 20
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            if (
                                bird_x + bird_radius > x
                                and bird_x - bird_radius < x + obstacle_width
                                and (bird_y - bird_radius < upper_height or bird_y + bird_radius > 480 - lower_height)
                            ):
                                game_over = True

                        # Incrementar puntos cuando el obstáculo haya pasado completamente
                        if obstacles and obstacles[0][0] + obstacle_width < bird_x - bird_radius:
                            if not bird_passed_obstacle:
                                score += 1
                                bird_passed_obstacle = True
                        else:
                            bird_passed_obstacle = False

                        cv2.circle(frame, (100, bird_y), 20, (0, 255, 0), -1)
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            cv2.rectangle(frame, (x, 0), (x + obstacle_width, upper_height), (0, 0, 255), -1)
                            cv2.rectangle(frame, (x, 480 - lower_height), (x + obstacle_width, 480), (0, 0, 255), -1)

                        # Incrementar velocidad cada 2 puntos
                        if score > 0 and score % 2 == 0 and obstacle_speed < max_speed:
                            obstacle_speed += obstacle_speed_increment

                    if not game_started:
                        cv2.putText(frame, "Press 'Space' to Start", (200, 200), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
                else:
                    cv2.putText(frame, "Game Over", (280, 200), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

                cv2.putText(frame, "Score: " + str(score), (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Cam', frame)

    key = cv2.waitKey(40)
    if key == 27:  # Tecla Esc para salir
        break
    elif key == 32:  # Tecla Espacio
        if not game_started:
            if not space_pressed:
                reset_game()
                game_started = True
                space_pressed = True
        else:
            space_pressed = False

cap.release()
cv2.destroyAllWindows()
                  


Camera 0


In [6]:
import cv2
import numpy as np
import time
import random

normalizatorHS = faceutils.Normalization()
FDet = FaceDetectors.FaceDetector()

font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap is not None:
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

cap.set(3, 640)
cap.set(4, 480)

imodoF = 0
imodoE = 0
score = 0
speed_increment_counter = 0
obstacle_speed = 5
obstacle_speed_increment = 1 # Aumento de velocidad cada 2 puntos
max_speed = 20  # Velocidad máxima de los obstáculos

bird_y = 240
bird_velocity = 0
gravity = 1
game_over = False
game_started = False
space_pressed = False
bird_passed_obstacle = False  # Variable para controlar el puntaje

obstacles = []

obstacle_width = 50  # Ancho de los obstáculos

def reset_game():
    global bird_y, bird_velocity, score, game_over, game_started, obstacles, speed_increment_counter, obstacle_speed
    bird_y = 240
    bird_velocity = 0
    score = 0
    game_over = False
    game_started = False
    obstacles = []
    speed_increment_counter = 0
    obstacle_speed = 5

def generate_obstacle():
    x = 640
    upper_height = random.randint(50, 250)
    lower_height = 350 - upper_height
    return (x, upper_height, lower_height)

reset_game()

while True:
    t = time.time()
    ret, frame = cap.read()
    B, G, R = cv2.split(frame)

    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values
        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

                if not game_over:
                    if game_started:
                        bird_y = ley

                        # Mover y actualizar los obstáculos existentes
                        new_obstacles = []
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            x -= obstacle_speed
                            if x >= -obstacle_width:
                                new_obstacles.append((x, upper_height, lower_height))
                        obstacles = new_obstacles

                        # Generar nuevos obstáculos
                        if len(obstacles) == 0 or obstacles[-1][0] < 400:  # Generar un nuevo obstáculo cuando el último esté cerca del centro
                            obstacles.append(generate_obstacle())

                        # Detección de colisión con los obstáculos
                        bird_x = 100
                        bird_radius = 20
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            if (
                                bird_x + bird_radius > x
                                and bird_x - bird_radius < x + obstacle_width
                                and (bird_y - bird_radius < upper_height or bird_y + bird_radius > 480 - lower_height)
                            ):
                                game_over = True

                        # Incrementar puntos cuando el obstáculo haya pasado completamente
                        if obstacles and obstacles[0][0] + obstacle_width < bird_x - bird_radius:
                            if not bird_passed_obstacle:
                                score += 1
                                bird_passed_obstacle = True
                        else:
                            bird_passed_obstacle = False

                        cv2.circle(frame, (100, bird_y), 20, (0, 255, 0), -1)
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            cv2.rectangle(frame, (x, 0), (x + obstacle_width, upper_height), (0, 0, 255), -1)
                            cv2.rectangle(frame, (x, 480 - lower_height), (x + obstacle_width, 480), (0, 0, 255), -1)

                        # Incrementar velocidad cada 2 puntos
                        if score > 0 and score % 2 == 0 and obstacle_speed < max_speed:
                            obstacle_speed += obstacle_speed_increment

                    if not game_started:
                        cv2.putText(frame, "Press 'Space' to Start", (200, 200), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
                else:
                    cv2.putText(frame, "Game Over", (280, 200), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

                cv2.putText(frame, "Score: " + str(score), (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Cam', frame)

    key = cv2.waitKey(40)
    if key == 27:  # Tecla Esc para salir
        break
    elif key == 32:  # Tecla Espacio
        if not game_started:
            if not space_pressed:
                reset_game()
                game_started = True
                space_pressed = True
        else:
            space_pressed = False

cap.release()
cv2.destroyAllWindows()
                  


Camera 0


In [6]:
import cv2
import numpy as np
import time
import random
import FaceNormalizationUtils as faceutils
import FaceDetectors

normalizatorHS = faceutils.Normalization()
FDet = FaceDetectors.FaceDetector()

font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap is not None:
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

cap.set(3, 640)
cap.set(4, 480)

imodoF = 0
imodoE = 0
score = 0
speed_increment_counter = 0
obstacle_speed = 5
obstacle_speed_increment = 1  # Aumento de velocidad cada 2 puntos
max_speed = 20  # Velocidad máxima de los obstáculos

bird_y = 240
bird_velocity = 0
gravity = 1
game_over = False
game_started = False
space_pressed = False
bird_passed_obstacle = False  # Variable para controlar el puntaje

# Cargar la imagen del avión
avion = cv2.imread('./images/AVION-removebg-preview.png', cv2.IMREAD_UNCHANGED)
# Asegurarse de que la imagen del avión tenga 4 canales (RGBA)
if avion.shape[2] == 4:
    # Crear una máscara para los píxeles transparentes
    mask = avion[:, :, 3]

    # Convertir la imagen del avión a un formato de 3 canales (BGR) y rellenar los píxeles transparentes con un color de fondo
    background_color = (255, 255, 255)  # Color de fondo deseado (blanco en este ejemplo)
    avion = cv2.cvtColor(avion, cv2.COLOR_BGRA2BGR)
    avion[mask != 255] = background_color

avion = cv2.resize(avion, (70, 60))
obstacles = []

obstacle_width = 50  # Ancho de los obstáculos

def reset_game():
    global bird_y, bird_velocity, score, game_over, game_started, obstacles, speed_increment_counter, obstacle_speed
    bird_y = 240
    bird_velocity = 0
    score = 0
    game_over = False
    game_started = False
    obstacles = []
    speed_increment_counter = 0
    obstacle_speed = 5

def generate_obstacle():
    x = 640
    upper_height = random.randint(50, 250)
    lower_height = 350 - upper_height
    return (x, upper_height, lower_height)

reset_game()

while True:
    t = time.time()
    ret, frame = cap.read()
    B, G, R = cv2.split(frame)

    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values
        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

                if not game_over:
                    if game_started:
                        bird_y = ley

                        # Mover y actualizar los obstáculos existentes
                        new_obstacles = []
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            x -= obstacle_speed
                            if x >= -obstacle_width:
                                new_obstacles.append((x, upper_height, lower_height))
                        obstacles = new_obstacles

                        # Generar nuevos obstáculos
                        if len(obstacles) == 0 or obstacles[-1][0] < 400:  # Generar un nuevo obstáculo cuando el último esté cerca del centro
                            obstacles.append(generate_obstacle())

                        # Detección de colisión con los obstáculos
                        bird_x = 100
                        bird_radius = 20
                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            if (
                                bird_x + bird_radius > x
                                and bird_x - bird_radius < x + obstacle_width
                                and (bird_y - bird_radius < upper_height or bird_y + bird_radius > 480 - lower_height)
                            ):
                                game_over = True

                        # Incrementar puntos cuando el obstáculo haya pasado completamente
                        if obstacles and obstacles[0][0] + obstacle_width < bird_x - bird_radius:
                            if not bird_passed_obstacle:
                                score += 1
                                bird_passed_obstacle = True
                        else:
                            bird_passed_obstacle = False

                        # Dibujar el avión en lugar de un círculo verde
                        frame[ley:ley+avion.shape[0], 100:100+avion.shape[1]] = avion

                        for obstacle in obstacles:
                            x, upper_height, lower_height = obstacle
                            cv2.rectangle(frame, (x, 0), (x + obstacle_width, upper_height), (0, 0, 255), -1)
                            cv2.rectangle(frame, (x, 480 - lower_height), (x + obstacle_width, 480), (0, 0, 255), -1)

                        # Incrementar velocidad cada 2 puntos
                        if score > 0 and score % 2 == 0 and obstacle_speed < max_speed:
                            obstacle_speed += obstacle_speed_increment

                    if not game_started:
                        cv2.putText(frame, "Press 'Space' to Start", (200, 200), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
                else:
                    cv2.putText(frame, "Game Over", (280, 200), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

                cv2.putText(frame, "Score: " + str(score), (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Cam', frame)

    key = cv2.waitKey(40)
    if key == 27:  # Tecla Esc para salir
        break
    elif key == 32:  # Tecla Espacio
        if not game_started:
            if not space_pressed:
                reset_game()
                game_started = True
                space_pressed = True
        else:
            space_pressed = False

cap.release()
cv2.destroyAllWindows()


Camera 0


In [9]:
import cv2
import numpy as np
import time
import random
import FaceNormalizationUtils as faceutils
import FaceDetectors

normalizatorHS = faceutils.Normalization()
FDet = FaceDetectors.FaceDetector()

font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap is not None:
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

cap.set(3, 640)
cap.set(4, 480)

imodoF = 0
imodoE = 0
score = 0
speed_increment_counter = 0
obstacle_speed = 5
obstacle_speed_increment = 1  # Aumento de velocidad cada 2 puntos
max_speed = 20  # Velocidad máxima de los obstáculos

bird_y = 240
bird_velocity = 0
gravity = 1
game_over = False
game_started = False
space_pressed = False
bird_passed_obstacle = False  # Variable para controlar el puntaje

# Cargar la imagen del avión
avion = cv2.imread('./images/AVION-removebg-preview.png', cv2.IMREAD_UNCHANGED)
# Asegurarse de que la imagen del avión tenga 4 canales (RGBA)
if avion.shape[2] == 4:
    # Crear una máscara para los píxeles transparentes
    mask = avion[:, :, 3]

    # Convertir la imagen del avión a un formato de 3 canales (BGR) y rellenar los píxeles transparentes con un color de fondo
    background_color = (255, 255, 255)  # Color de fondo deseado (blanco en este ejemplo)
    avion = cv2.cvtColor(avion, cv2.COLOR_BGRA2BGR)
    avion[mask != 255] = background_color

avion = cv2.resize(avion, (70, 60))
obstacles = []

obstacle_width = 50  # Ancho de los obstáculos

def reset_game():
    global bird_y, bird_velocity, score, game_over, game_started, obstacles, speed_increment_counter, obstacle_speed
    bird_y = 240
    bird_velocity = 0
    score = 0
    game_over = False
    game_started = False
    obstacles = []
    speed_increment_counter = 0
    obstacle_speed = 5

def generate_obstacle():
    x = 640
    upper_height = random.randint(50, 250)
    lower_height = 350 - upper_height
    return (x, upper_height, lower_height)

reset_game()
while True:
    t = time.time()
    ret, frame = cap.read()
    B, G, R = cv2.split(frame)

    # Crear un fondo azul
    blue_bg = np.zeros_like(frame)
    blue_bg[:, :] = (255, 0, 0)  # Color azul

    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values
        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(blue_bg, (x, y), (x + w, y + h), (0, 0, 255), 2)
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                for (x, y) in shape:
                    cv2.circle(blue_bg, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(blue_bg, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(blue_bg, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

    # Combinar el fondo azul con la imagen de la cámara
    frame = cv2.addWeighted(frame, 1, blue_bg, 0.5, 0)

    if not game_over:
        if game_started:
            bird_y = ley

            # Mover y actualizar los obstáculos existentes
            new_obstacles = []
            for obstacle in obstacles:
                x, upper_height, lower_height = obstacle
                x -= obstacle_speed
                if x >= -obstacle_width:
                    new_obstacles.append((x, upper_height, lower_height))
            obstacles = new_obstacles

            # Generar nuevos obstáculos
            if len(obstacles) == 0 or obstacles[-1][0] < 400:  # Generar un nuevo obstáculo cuando el último esté cerca del centro
                obstacles.append(generate_obstacle())

            # Detección de colisión con los obstáculos
            bird_x = 100
            bird_radius = 20
            for obstacle in obstacles:
                x, upper_height, lower_height = obstacle
                if (
                    bird_x + bird_radius > x
                    and bird_x - bird_radius < x + obstacle_width
                    and (bird_y - bird_radius < upper_height or bird_y + bird_radius > 480 - lower_height)
                ):
                    game_over = True

            # Incrementar puntos cuando el obstáculo haya pasado completamente
            if obstacles and obstacles[0][0] + obstacle_width < bird_x - bird_radius:
                if not bird_passed_obstacle:
                    score += 1
                    bird_passed_obstacle = True
            else:
                bird_passed_obstacle = False

            # Dibujar el avión en lugar de un círculo verde
            frame[ley:ley + avion.shape[0], 100:100 + avion.shape[1]] = avion

            for obstacle in obstacles:
                x, upper_height, lower_height = obstacle
                cv2.rectangle(frame, (x, 0), (x + obstacle_width, upper_height), (0, 0, 255), -1)
                cv2.rectangle(frame, (x, 480 - lower_height), (x + obstacle_width, 480), (0, 0, 255), -1)

            # Incrementar velocidad cada 2 puntos
            if score > 0 and score % 2 == 0 and obstacle_speed < max_speed:
                obstacle_speed += obstacle_speed_increment

        if not game_started:
            cv2.putText(frame, "Press 'Space' to Start", (200, 200), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    else:
        cv2.putText(frame, "Game Over", (280, 200), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.putText(frame, "Score: " + str(score), (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Cam', frame)

    key = cv2.waitKey(40)
    if key == 27:  # Tecla Esc para salir
        break
    elif key == 32:  # Tecla Espacio
        if not game_started:
            if not space_pressed:
                reset_game()
                game_started = True
                space_pressed = True
        else:
            space_pressed = False

cap.release()
cv2.destroyAllWindows()


Camera 0


In [14]:
import cv2
import numpy as np
import time
import random
import FaceNormalizationUtils as faceutils
import FaceDetectors

normalizatorHS = faceutils.Normalization()
FDet = FaceDetectors.FaceDetector()

font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap is not None:
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

cap.set(3, 640)
cap.set(4, 480)

imodoF = 0
imodoE = 0
score = 0
speed_increment_counter = 0
obstacle_speed = 5
obstacle_speed_increment = 1  # Aumento de velocidad cada 2 puntos
max_speed = 20  # Velocidad máxima de los obstáculos

bird_y = 240
bird_velocity = 0
gravity = 1
game_over = False
game_started = False
space_pressed = False
bird_passed_obstacle = False  # Variable para controlar el puntaje

# Crear un fondo azul sólido
blue_bg = np.zeros((480, 640, 3), dtype=np.uint8)
blue_bg[:, :] = (255, 0, 0)  # Color azul

# Cargar la imagen del avión
avion = cv2.imread('./images/AVION-removebg-preview.png', cv2.IMREAD_UNCHANGED)
# Asegurarse de que la imagen del avión tenga 4 canales (RGBA)
if avion.shape[2] == 4:
    # Crear una máscara para los píxeles transparentes
    mask = avion[:, :, 3]

    # Convertir la imagen del avión a un formato de 3 canales (BGR) y rellenar los píxeles transparentes con un color de fondo
    background_color = (255, 0, 0)  # Color de fondo deseado (blanco en este ejemplo)
    avion = cv2.cvtColor(avion, cv2.COLOR_BGRA2BGR)
    avion[mask != 255] = background_color

avion = cv2.resize(avion, (70, 60))
obstacles = []

obstacle_width = 50  # Ancho de los obstáculos

def reset_game():
    global bird_y, bird_velocity, score, game_over, game_started, obstacles, speed_increment_counter, obstacle_speed
    bird_y = 240
    bird_velocity = 0
    score = 0
    game_over = False
    game_started = False
    obstacles = []
    speed_increment_counter = 0
    obstacle_speed = 5

def generate_obstacle():
    x = 640
    upper_height = random.randint(50, 250)
    lower_height = 350 - upper_height
    return (x, upper_height, lower_height)

reset_game()

while True:
    t = time.time()
    ret, frame = cap.read()
    B, G, R = cv2.split(frame)

    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values
        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

    # Combinar el fondo azul con la imagen de la c
    # Combinar el fondo azul con la imagen de la claridad
    frame = blue_bg.copy()

    if not game_over:
        if game_started:
            bird_y = ley

            # Mover y actualizar los obstáculos existentes
            new_obstacles = []
            for obstacle in obstacles:
                x, upper_height, lower_height = obstacle
                x -= obstacle_speed
                if x >= -obstacle_width:
                    new_obstacles.append((x, upper_height, lower_height))
            obstacles = new_obstacles

            # Generar nuevos obstáculos
            if len(obstacles) == 0 or obstacles[-1][0] < 400:  # Generar un nuevo obstáculo cuando el último esté cerca del centro
                obstacles.append(generate_obstacle())

            # Detección de colisión con los obstáculos
            bird_x = 100
            bird_radius = 20
            for obstacle in obstacles:
                x, upper_height, lower_height = obstacle
                if (
                    bird_x + bird_radius > x
                    and bird_x - bird_radius < x + obstacle_width
                    and (bird_y - bird_radius < upper_height or bird_y + bird_radius > 480 - lower_height)
                ):
                    game_over = True

            # Incrementar puntos cuando el obstáculo haya pasado completamente
            if obstacles and obstacles[0][0] + obstacle_width < bird_x - bird_radius:
                if not bird_passed_obstacle:
                    score += 1
                    bird_passed_obstacle = True
            else:
                bird_passed_obstacle = False

            # Dibujar el avión en lugar de un círculo verde
            frame[ley:ley+avion.shape[0], 100:100+avion.shape[1]] = avion

            for obstacle in obstacles:
                x, upper_height, lower_height = obstacle
                cv2.rectangle(frame, (x, 0), (x + obstacle_width, upper_height), (0, 0, 255), -1)
                cv2.rectangle(frame, (x, 480 - lower_height), (x + obstacle_width, 480), (0, 0, 255), -1)

            # Incrementar velocidad cada 2 puntos
            if score > 0 and score % 2 == 0 and obstacle_speed < max_speed:
                obstacle_speed += obstacle_speed_increment

        if not game_started:
            cv2.putText(frame, "Press 'Space' to Start", (200, 200), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    else:
        cv2.putText(frame, "Game Over", (280, 200), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.putText(frame, "Score: " + str(score), (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Cam', frame)

    key = cv2.waitKey(40)
    if key == 27:  # Tecla Esc para salir
        break
    elif key == 32:  # Tecla Espacio
        if not game_started:
            if not space_pressed:
                reset_game()
                game_started = True
                space_pressed = True
        else:
            space_pressed = False

cap.release()
cv2.destroyAllWindows()


Camera 0
